# DATA PREPARATION (Fake user creation)

Création de faux utilissateurs pour tester nos models

*USER :*
- User_1001 JAZZ -->last_fm_fake_user(1001)_jazz.csv
- User_1002 CLASSIC -->last_fm_fake_user(1002)_classic.csv
- User_1003 POP -->last_fm_fake_user(1003)_pop.csv
- User_1004 ROCK -->last_fm_fake_user(1004)_rock.csv
- User_1005 RAP -->last_fm_fake_user(1005)_rap.csv

## Imports

In [5]:
import pandas as pd

import ipywidgets as widgets
import datetime as dt

## Data import 

In [3]:
df = pd.read_csv('filtered data/filtered_fm_dataset.csv', index_col=0)

C:\Users\Alumet\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
df.head()

,user_id,time_stamp,artist_id,artist_name,track_id,track_name
17,1,2009-05-03T15:10:18Z,4,Minus 8,0,Elysian Fields
18,1,2009-05-03T15:04:31Z,5,Beanfield,1,Planetary Deadlock
20,1,2009-05-03T14:50:51Z,7,Alif Tree,2,Deadly Species
21,1,2009-05-03T14:46:29Z,4,Minus 8,3,Cold Fusion
43,1,2009-05-02T15:00:59Z,14,4Hero,4,Look Inside


## Widget de creation d'utilisateur

In [6]:
class play_list_creator():
    
    def __init__(self, df):
        
        
        self.df_sub = df.groupby(['track_id']).first().reset_index()[['track_id','artist_id','artist_name','track_name']]
        
        self._reset_result() # initialisation du DataFrame de resultat
        
        self._creat_artist_widjet() # widget de selection d'artiste
        self._creat_track_widjet() # widget de selection de titre
        self._creat_button_widjet() # bouton d'ajout
        
        self.display()
    
    # Reset des resultats
    def _reset_result(self):
        
        self.result=pd.DataFrame({'time_stamp':[] ,'track_id':[], 'artist_id':[], 'artist_name':[], 'track_name':[]})
        
        self.result['track_id'] = self.result['track_id'].astype('int')
        self.result['artist_id'] = self.result['artist_id'].astype('int')
    
    # Widget de selection d'artiste. liste dropdown avec autocompletion
    def _creat_artist_widjet(self):
        
        options = [x for x in self.df_sub['artist_name'].unique()]
        self.artist_w = widgets.Combobox(
                        laceholder='Choose an artist',
                        options= options,
                        description='Artist:',
                        ensure_option=True,
                        disabled=False
                        )
        
        self.artist_w.observe(self._on_artist_change)
    
    # Widget de selection de titre. liste dropdown avec autocompletion
    def _creat_track_widjet(self):

        self.track_w = widgets.Combobox(
                       placeholder='Choose a track',
                       options= [],
                       description='Track:',
                       ensure_option=True,
                       disabled=False
                       )
    
    # Quand un artiste change on change la liste des titres par ceux qui lui sont associés
    def _on_artist_change(self, change):
        if change['type'] == 'change' and change['name'] == 'value':
            self.track_w.options=[x for x in self.df_sub[self.df_sub['artist_name']==self.artist_w.value]['track_name'].unique()]
            self.track_w.value=''
    
    # Widget Bouton d'ajout de la selection aux resultats
    def _creat_button_widjet(self):
        
        self.button = widgets.Button(description="Add to list!")
        self.button.on_click(self._on_button_clicked)
    
    # Quand le bouton est cliqué on ajoute la selection aux resultats
    def _on_button_clicked(self, change):
        
        temp = self.df_sub[(self.df_sub['artist_name']==self.artist_w.value) & (self.df_sub['track_name']==self.track_w.value)].copy()
        
        time = dt.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
        temp['time_stamp']=[time]
        
        if temp['track_id'].values[0] not in self.result['track_id'].unique(): # pas de doublons
            self.result = pd.concat([self.result,temp])

    # Afficher le widget
    def display(self):
        display(self.artist_w)
        display(self.track_w)
        display(self.button)
        

# Playlist Creator

In [7]:
PC = play_list_creator(df)

Combobox(value='', description='Artist:', ensure_option=True, options=('Minus 8', 'Beanfield', 'Alif Tree', '4…

Combobox(value='', description='Track:', ensure_option=True, placeholder='Choose a track')

Button(description='Add to list!', style=ButtonStyle())

In [9]:
PC.result

,time_stamp,track_id,artist_id,artist_name,track_name
4848,2022-04-18T10:25:08Z,4848,1564,Nirvana,Smells Like Teen Spirit
11536,2022-04-18T10:25:24Z,11536,1982,Nirvana,Love Buzz
9968,2022-04-18T10:25:29Z,9968,1982,Nirvana,Come As You Are
6440,2022-04-18T10:25:42Z,6440,1731,Ac/Dc,Big Balls
20563,2022-04-18T10:25:48Z,20563,1731,Ac/Dc,Stormy May Day
20831,2022-04-18T10:25:52Z,20831,1731,Ac/Dc,Let There Be Rock
22093,2022-04-18T10:25:58Z,22093,4727,Janis Joplin,Piece Of My Heart
62149,2022-04-18T10:26:02Z,62149,4727,Janis Joplin,Half Moon


In [9]:
#PC._reset_result()

## Sauvegarde

In [10]:
df_rock = PC.result.copy()

In [11]:
df_rock.reset_index(inplace=True)
df_rock.rename(columns={'index':'user_id'}, inplace=True)
df_rock['user_id']=1004

In [13]:
df_rock.to_csv('filtered data/fake_user/last_fm_fake_user(1004)_rock.csv')